In [ ]:
import pandas as pd
from newsapi import NewsApiClient
from transformers import pipeline
import torch
import os
from tqdm import tqdm

# Initialize News API
newsapi = NewsApiClient(api_key='fe27f943772a4235a29f751edabae735')

# Fetch Bitcoin news
all_articles = newsapi.get_everything(q='Bitcoin', language='en', sort_by='publishedAt', page_size=100)
news_df = pd.DataFrame(all_articles['articles'])
news_df['publishedAt'] = pd.to_datetime(news_df['publishedAt']).dt.normalize()

# Initialize LLM sentiment pipeline 
sentiment_pipeline = pipeline(
    "text-classification", 
    model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis",
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

def analyze_sentiment(text):
    try:
        result = sentiment_pipeline(text[:512])[0]  # Truncate to model max length
        return result['score'] * (1 if result['label'] == 'positive' else -1)
    except:
        return 0  # Neutral if error

# Process headlines with progress bar
tqdm.pandas()
news_df['llm_sentiment'] = news_df['title'].progress_apply(analyze_sentiment)

# Save results
news_df[['publishedAt', 'llm_sentiment']].to_csv('llm_sentiment_scores.csv', index=False)
print("Sentiment analysis completed and saved.")

NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [2]:
import pandas as pd
# Load your existing file
df = pd.read_csv('crypto_sentiment_results.csv')

# Extract and display dates
print("\n=== Dates in Saved Data ===")
dates = pd.to_datetime(df['publishedAt']).dt.date.unique()
for date in sorted(dates):
    print(date)


=== Dates in Saved Data ===
2025-05-27
2025-05-28
